In [6]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d


# B92 Involute Spline Optimizer

Written in the same format as shaft optimizer  

This only factors in root stresses of splines. Validate tubes by simplifying splines to the minimum diameter and the ID of the tube.   

All splines are assumed to be Fillet root with a side fit

In [7]:
#TODO add tensile strength

materials = { # all stresses are in lb-ft^2, density lb/in^3
    'al7075' : {
        "material_name": 'al7075',
        "shear_stress": 6912000,
        "density": .102,
    },
    'al6061' : {
        "material_name": 'al6061',
        "shear_stress": 4320000,
        "density": .0975,
    },
    '4340_annealed' : {
        "material_name": '4340_annealed',
        "shear_stress": 9072000,
        "density": .284,
    },
   '4340_normalized' : {
        "material_name": '4340_normalized',
        "shear_stress": 15840000,
        "density": .284,
    }
}

In [8]:
# tooth geometry

def get_pitch_diameter(N, P):
    return N/P

def get_tooth_height(N, P, φ): # major - minor diameter
    D_o = (N + 1)/P

    #for bmt (and my) sanity pressure angle will be assumed to be 30 degrees
    if P < 12 and P > 2.5:
        D_r = (N - 1.8)/P
    elif P < 16:
        D_r = (N - 2)/P
    else:
        D_r = (N - 2.3)/P
    
    return D_o - D_r
    

def get_tooth_thickness(P):
    return math.pi/(2*P)

def max_effective_length(pitch_diameter): # line fit for fixed splines with helix modification
    return -1 + 7/5*pitch_diameter

SyntaxError: expected ':' (1832050991.py, line 23)

*All tooth geometry is a function of pressure angle, pitch diameter, and number of teeth. Equations are pulled from BDM-1550.*

In [ ]:
#methods from shaft optimizer
def get_torsional_torque(shear_stress, thickness, diameter):
    t = thickness/12
    d = diameter/12
    
    return (math.pi/16) * shear_stress * (d**4 - (d - t)**4)/d

def get_max_id(shear_stress, max_torque, diameter):
    return ((max_torque/shear_stress)*(16/math.pi)*diameter - diameter**4)**(1/4)


def get_tube_planar_mass(thickness, diameter, density):
    return (math.pi * (diameter/2)**2 - (math.pi * (diameter/2 - thickness)**2)) * density

def compressive_stress(N,L,H,D, S_c, K_m, K_l, K_o, K_t): #compressive stress for fixed splines
    
    teeth_parameters = N*L*H*D

    return ((S_c * teeth_parameters)/(2*K_m)) * (K_l/(K_o*K_t))

def shear_stress(N,L,t,D, K_m, S_sa, K_l, K_o):
    teeth_parameters = N*L*t*D

    return S_sa * teeth_parameters * (K_l/K_o) * (1/(2*K_m))

def get_max_torque(N,L,H,D,id, S_c, S_sa, K_m, K_l, K_o, K_t):
    max_shear_torque = shear_stress(N,L,t,D, K_m, S_sa, K_l, K_o)
    max_compressive_torque = compressive_stress(N,L,H,D, S_c, K_m, K_l, K_o, K_t)
    max_torsional_torque = get_torsional_torque(S_sa, D - id, D)

    return math.min(max_shear_torque, max_compressive_torque, max_torsional_torque)


### Parameters

**S_c** - compressive stress (psi)  
**S_s** - shear stress (psi)  
**S_sa** - allowable shear stress (psi)  
**S_ca** - allowable contact stress (psi) ~ generalized to compressive stress  

**T** - torque (in-lb)  

**D** - pitch diameter (in)  
**N** - number of teeth  
**L** - spline length  
**H** - tooth height   
**t** - tooth thickness (in)  
**φ** - Pressure Angle  
**P** - Pitch

**K_m** - load distribution factor (fraction of splines engaged)  
**K_l** - life factor  
**K_o** - overload factor  
**K_t** - spline type factor  



In [ ]:
def NPplot(material, zaxis):
    ax = plt.axes(projection='3d')

    zdata = material[zaxis]
    xdata = material["N"]
    ydata = material["P"]
    
    ax.set_xlabel('N')
    ax.set_ylabel('P')
    ax.set_zlabel(zaxis)
    
    ax.set_ylim(0,.05)

    ax.scatter3D(xdata, ydata, zdata, c=zdata)

    plt.title(material["material_name"])
    plt.show()

In [ ]:
#Constants


K_m = 0.5
K_l = 1
K_o = 1
K_t = 1

L = 1 

pitch_selection = [2.5,3,4,5,6,8,10,12,16,20,24,32,40,48]

inp_torque = 300


In [10]:
for material_name, material in materials.items():
    material["P"] = []
    material["N"] = []
    material["max_torque"] = []
    material["planar_mass"] = []
    material["id"] = []

    shear_stress = material["shear_stress"]
    tensile_stress = material["tensile_stress"]

    # compressive_stress(N,L,H,D, S_c, K_m, K_l, K_o, K_t)

    for P in pitch_selection: 
        for N in range(6,60):
            D = get_pitch_diameter(N, P)
            H = get_tooth_height(N, P, 30)
            t = get_tooth_thickness(P)

            id = get_max_id(shear_stress, inp_torque, D)
            T = get_max_torque(N,L,H,D,id,material["tensile_stress"], material["shear_stress"], K_m, K_l, K_o, K_t)
            planar_mass = get_tube_planar_mass(D - id, D, material["density"])
            
            if (T > inp_torque) or (max_effective_length(D) < L): # TODO: generate graphs without max effective length to see difference
                material["max_torque"].append(T)
                material["id"].append(id)
                material["planar_mass"].append(planar_mass)
    
    
    NPplot(material, "max_torque") # TODO write max_torque planar_mass cost function
    NPplot(material, "planar_mass")
            

KeyError: 'tensile_stress'